# Splunk

### !!! Работает с Python 2 и Python 3 !!!

In [11]:
from time import sleep
import sys
import splunklib
import splunklib.client as client
import splunklib.results as results

import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import datetime as dt

In [2]:
log_level = 0

def outLog(message, level=1):
    if(level <= log_level):
        print (message)
        sys.stdout.flush()

### Подключение к Splunk

In [4]:
#HOST = "localhost"
HOST = "splunk-search01.roamability.com"
PORT = 8089
USERNAME = "pavel"
PASSWORD = "O777ver7k1"

# Create a Service instance and log in 
service = client.connect(
    host=HOST,
    port=PORT,
    username=USERNAME,
    password=PASSWORD)

# Print installed apps to the console to verify login
for app in service.apps:
    print(app.name)

alert_logevent
alert_webhook
appsbrowser
framework
gettingstarted
introspection_generator_addon
launcher
learned
legacy
lookup_editor
sample_app
search
simple_xml_examples
splunk_app_db_connect
splunk_archiver
splunk_datasets_addon
splunk_httpinput
splunk_instrumentation
splunk_monitoring_console
SplunkForwarder
SplunkLightForwarder


### Использование сохраненных запросов и конструкции
job = mysavedsearch.dispatch()<br>
jobresults = job.results()<br>
jobresults.read(1024)

In [7]:
#Посмотреть сохраненные запросы по заданному шаблону

#List the saved searches that are available to the current user

q_search='Test' #Задаем вхождение в искомый запрос

savedsearches = service.saved_searches
for savedsearch in savedsearches:
    if savedsearch.name.find(q_search)!=-1:
        print("Search name: '" + savedsearch.name + "'") #Print search name
        print("Query:\n" + savedsearch["search"] + "\n") #Print query of the search

Search name: 'Test Search'
Query:
index=sdr PROTOCOL=MAP OPCODE=2 SERVICE="stp" | head 1 | table _time,IN_IMSI,OPCODE,O_GT,D_GT,RESULT



In [8]:
# Retrieve the result with saved search

search_name="Test Search"
mysavedsearch = service.saved_searches[search_name]

# Run the saved search
job = mysavedsearch.dispatch()
# Create a small delay to allow time for the update between server and client
sleep(2)
# Wait for the job to finish--poll for completion and display stats
while True:
    job.refresh()
    stats = {"isDone": job["isDone"],
             "doneProgress": float(job["doneProgress"])*100,
              "scanCount": int(job["scanCount"]),
              "eventCount": int(job["eventCount"]),
              "resultCount": int(job["resultCount"])}
    status = ("\r%(doneProgress)03.1f%%   %(scanCount)d scanned   "
              "%(eventCount)d matched   %(resultCount)d results") % stats
    sys.stdout.write(status)
    sys.stdout.flush()
    if stats["isDone"] == "1":
        break
    sleep(2)
# Display the search results now that the job is done
jobresults = job.results()
while True:
    content = jobresults.read(1024)
    if len(content) == 0: break
    sys.stdout.write(content)
    sys.stdout.flush()
sys.stdout.write("\n")

100.0%   107 scanned   1 matched   1 results<?xml version='1.0' encoding='UTF-8'?>
<results preview='0'>
<meta>
<fieldOrder>
<field>_time</field>
<field>IN_IMSI</field>
<field>OPCODE</field>
<field>O_GT</field>
<field>D_GT</field>
<field>RESULT</field>
</fieldOrder>
</meta>
	<result offset='0'>
		<field k='_time'>
			<value><text>2018-05-11T07:30:41.000+02:00</text></value>
		</field>
		<field k='IN_IMSI'>
			<value><text>425019612523469</text></value>
		</field>
		<field k='OPCODE'>
			<value><text>2</text></value>
		</field>
		<field k='O_GT'>
			<value><text>41794943000</text></value>
		</field>
		<field k='D_GT'>
			<value><text>972549612523469</text></value>
		</field>
		<field k='RESULT'>
			<value><text>T_Error</text></value>
		</field>
	</result>
</results>


### Выгрузка в DF с использованием .ResultsReader

In [13]:
#Простеейшая конструкция для примера

def FormatSplunkEngineDate(dt): return dt.strftime("%Y-%m-%dT%H:%M:%S")

dateFrom = dt.datetime.strptime('2018-01-12T05:00:00', '%Y-%m-%dT%H:%M:%S')
dateTo = dt.datetime.strptime('2018-01-12T12:00:00', '%Y-%m-%dT%H:%M:%S')

kwargs_export = {"earliest_time": FormatSplunkEngineDate(dateFrom),
                "latest_time":FormatSplunkEngineDate(dateTo) ,
                "search_mode": "normal",
                #"output_mode":"JSON",
                #"rf" : fields,
                "supports_multivalues":True
                }
searchquery_export = 'search index=sdr PROTOCOL=MAP OPCODE=2 SERVICE="stp" | head 1 | table _time,IN_IMSI,OPCODE,O_GT,D_GT,RESULT'
exportsearch_results = service.jobs.export(searchquery_export, **kwargs_export)
reader = results.ResultsReader(exportsearch_results)
for result in reader:
    print(result)

DEBUG: Configuration initialization for /opt/splunk/etc took 16ms when dispatching a search (search ID: 1526016777.171205)
DEBUG: base lispy: [ AND 2 index::sdr map stp ]
DEBUG: search context: user="pavel", app="search", bs-pathname="/opt/splunk/etc"
WARN: Unable to distribute to peer named 172.19.11.50:8089 at uri=172.19.11.50:8089 using the uri-scheme=https because peer has status="Down".  Please verify uri-scheme, connectivity to the search peer, that the search peer is up, and an adequate level of system resources are available. See the Troubleshooting Manual for more information.
OrderedDict([('_time', '2018-01-12 11:59:59.000 CET'), ('IN_IMSI', '425019612056086'), ('OPCODE', '2'), ('O_GT', '393205981000'), ('D_GT', '972549612056086'), ('RESULT', 'ok')])


In [15]:
#Пример выгрузки

#Создаем необходимые процедуры

def FormatSplunkEngineDate(dt): return dt.strftime("%Y-%m-%dT%H:%M:%S")

def LoadSplunkResult(exportsearch_results):
        outputResult = list()
        
        i=0
    # Get the results and store them
        reader = results.ResultsReader(exportsearch_results)
        for result in reader:   
            if isinstance(result, dict):
                outputResult.append(result)
                i+=1
                if i%100==0: outLog("loaded {} records from splunk".format(i),0)
            elif isinstance(result, results.Message):
                # Diagnostic messages may be returned in the results
                outLog("Message: %s" % result)
        outLog("Splunk search found {} messages".format(len(outputResult)),0)
        return outputResult

def getSplunkFileResult(service, dateFrom, dateTo, searchquery_export, fileMask=None,protocol=None, bEchoTime = False):
    # Run an export search and display the results using the results reader.
    # Set the parameters for the search:
    sfrom = dateFrom if  bEchoTime else FormatSplunkEngineDate(dateFrom) 
    sto = dateTo if  bEchoTime else FormatSplunkEngineDate(dateTo)
    kwargs_export = {"earliest_time": sfrom,
                     "latest_time":sto ,
                     "search_mode": "normal",
                     #"output_mode":"JSON",
                     #"rf" : fields,
                     "supports_multivalues":True
                     }
    outLog("Run Splunk file search by query {}".format(searchquery_export),0)

    exportsearch_results = service.jobs.export(searchquery_export, **kwargs_export)

    return LoadSplunkResult(exportsearch_results)

In [17]:
#Указываем время, запрос и запускаем запрос

dateFrom = dt.datetime.strptime('2018-01-12T05:00:00', '%Y-%m-%dT%H:%M:%S')
dateTo = dt.datetime.strptime('2018-01-12T12:00:00', '%Y-%m-%dT%H:%M:%S')
searchquery_export =\
'''
search index=sdr PROTOCOL=MAP OPCODE=2 SERVICE="stp" | head 15 | table _time,IN_IMSI,OPCODE,O_GT,D_GT,RESULT
'''

SplunkFileResult = getSplunkFileResult(service, dateFrom, dateTo, searchquery_export, fileMask=None,protocol=None, bEchoTime = False)

Run Splunk file search by query 
search index=sdr PROTOCOL=MAP OPCODE=2 SERVICE="stp" | head 15 | table _time,IN_IMSI,OPCODE,O_GT,D_GT,RESULT

Splunk search found 15 messages


In [18]:
#Выгружаем результат в DataFrame

#!!! В данном варианте не выгрвжаются все данные.
# Много строк с нулями если делать длительный агрегированный запрос

df=DataFrame(SplunkFileResult)
df.head()

,_time,IN_IMSI,OPCODE,O_GT,D_GT,RESULT
0,2018-01-12 11:59:59.000 CET,425019612056086,2,393205981000,972549612056086,ok
1,2018-01-12 11:59:59.000 CET,425019610393162,2,886935420431,972549610393162,T_Error
2,2018-01-12 11:59:59.000 CET,425019629501277,2,3584097020,972549629501277,ok
3,2018-01-12 11:59:59.000 CET,260060140000621,2,14047259687,487900140000621,ok
4,2018-01-12 11:59:59.000 CET,425019610603034,2,33660001066,972549610603034,ok


In [19]:
## Исправленный вариант на основе:
#http://docs.splunk.com/DocumentationStatic/PythonSDK/1.6.3/client.html#splunklib.client.Job.results
#Ждет завершения выгрузки.

#Указываем время, запрос и запускаем запрос

dateFrom = dt.datetime.strptime('2018-04-12T08:00:00', '%Y-%m-%dT%H:%M:%S')
dateTo = dt.datetime.strptime('2018-04-13T08:00:00', '%Y-%m-%dT%H:%M:%S')
searchquery_export =\
'''
search index=sdr (4250196* OR 26006*) PROTOCOL=S6a ANS_RES_CODE=2001
| eval CODE_TEMP=case(isnotnull(ANS_RES_CODE),ANS_RES_CODE,isnull(ANS_RES_CODE), ANS_EXP_RES_CODE, isnull(ANS_EXP_RES_CODE), EXP_RES_CODE, isnull(EXP_RES_CODE), RES_CODE)
| eval CODE=if(isnotnull(CODE_TEMP), CODE_TEMP, "PUR_NOR") | eval CUSTOMER=coalesce(THIRD_PARTY,MVNO)
| Timechart span=15m count by CUSTOMER  | table *
'''

def FormatSplunkEngineDate(dt): return dt.strftime("%Y-%m-%dT%H:%M:%S")
bEchoTime = False

sfrom = dateFrom if  bEchoTime else FormatSplunkEngineDate(dateFrom) 
sto = dateTo if  bEchoTime else FormatSplunkEngineDate(dateTo)
kwargs_export = {"earliest_time": sfrom,
                 "latest_time":sto ,
                 "search_mode": "normal",
                 #"output_mode":"JSON",
                 #"rf" : fields,
                 "supports_multivalues":True
                 }

job = service.jobs.create(searchquery_export, **kwargs_export)
while not job.is_done():
    sleep(.2)
rr = results.ResultsReader(job.results())

outputResult = list()
i=0

for result in rr:
            if isinstance(result, dict):
                outputResult.append(result)
assert rr.is_preview == False

df=DataFrame(outputResult)
df.tail()

,C9,MB,Mondicon,NAKA,OTHER,PuertoSeguro,Rewicom,Roamability,Telzar,TinkLab,Tinklabs,_span,_time
91,636,405,144,1566,0,14,0,170,2135,0,558,900,2018-04-13T06:45:00.000+02:00
92,842,458,160,1626,0,3,1,167,2238,4,3643,900,2018-04-13T07:00:00.000+02:00
93,803,272,151,1728,0,9,1,164,2247,0,745,900,2018-04-13T07:15:00.000+02:00
94,851,408,147,1841,0,13,0,168,2241,0,468,900,2018-04-13T07:30:00.000+02:00
95,931,463,123,1842,0,27,0,171,2347,0,226,900,2018-04-13T07:45:00.000+02:00
